# تلخيص النصوص باستعمال النماذج اللغوية الكبيرة

In [1]:
%pip install -q -U pydantic
%pip install -q anthropic

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3

In [3]:
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-west-2")
bedrock = boto3.client("bedrock", region_name="us-west-2")

In [4]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '2174eb0e-f68f-4c75-bbdc-ee1d079ca5ed',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 04 Sep 2024 12:02:43 GMT',
   'content-type': 'application/json',
   'content-length': '28476',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2174eb0e-f68f-4c75-bbdc-ee1d079ca5ed'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-g1-text-02',
   'modelId': 'amazon.titan-embed-g1-text-02',
   'modelName': 'Titan Text Embeddings v2',
   'providerName': 'Amazon',
   'inp

In [5]:
file_path = "../output/cleaned/aws-cloud-adoption-framework.txt"

In [6]:

!head -20 {file_path}

إطار عمل اعتماد السحابة من AWS
التحول الرقمي المدعوم بالسحابة
ّ     تسريع عمليات



تم نشره ألول مرة في فبراير/شباط 2015

تم التحديث في  22نوفمبر /تشرين الثاني 2021
اإلشعارات
العمالء مسؤولون عن إجراء تقييمهم المستقل للمعلومات الموجودة في هذا المستند .هذا المستند( :أ) ألغراض إعالمية فقط،
(ب) ويمثل عروض منتجات  AWSوممارساتها الحالية التي تخضع للتغيير بدون إشعار( ،ج) وال يشكِّل أي التزامات أو
ضمانات من جانب  AWSوالشركات التابعة لها أو مورديها أو مرخصيها .يتم توفير منتجات  AWSأو خدماتها «كما
هي» دون ضمانات أو إقرارات أو شروط من أي نوع ،سواء كانت صريحة أو ضمنية .وتخضع مسؤوليات AWS
والتزاماتها نحو عمالئها إلى اتفاقيات  ،AWSوال يمثل هذا المستند جز ًءا من أي اتفاقية مبرمة بين  AWSوعمالئها وال
ً
تعديال لها. يمثل





In [7]:
with open(file_path, "r") as doc:
    document = doc.read()

In [8]:
print(document[:200])

إطار عمل اعتماد السحابة من AWS
التحول الرقمي المدعوم بالسحابة
ّ     تسريع عمليات



تم نشره ألول مرة في فبراير/شباط 2015

تم التحديث في  22نوفمبر /تشرين الثاني 2021
اإلشعارات
العمالء مسؤولون عن إجراء 


In [9]:
import anthropic
anthropic_client = anthropic.Anthropic()

In [13]:
anthropic_client.count_tokens(document)

51600

In [14]:
prompt_data = """
Review the following Arabic document carrefully, then summarize it. Please provide a concise yet comprehensive summary that captures the main points and key details of the document.
Your summary should be written in clear and coherent Arabic. Write the summary inside a <summary> xml tag.

Document: 

<document>
{document}
</document>
"""

In [15]:
import json

body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 4096,
    "temperature": 0.1,
    "top_k":250,
    "top_p":0.99,
    "messages": [
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt_data.format(document=document)}]
        }
    ],
})

In [17]:
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())


In [20]:
type(response_body), response_body.keys()

(dict,
 dict_keys(['id', 'type', 'role', 'model', 'content', 'stop_reason', 'stop_sequence', 'usage']))

In [23]:
print(response_body["content"][0]["text"])

Voici un résumé concis du document en arabe :

<summary>
يقدم إطار عمل اعتماد السحابة من AWS (AWS CAF) إرشادات حول أفضل الممارسات لمساعدة المؤسسات على التحول الرقمي باستخدام خدمات AWS. يحدد الإطار 6 مناظير رئيسية (الأعمال، الأشخاص، الحوكمة، المنصات، الأمان، العمليات) و إمكانيات تأسيسية تحت كل منظور لتقييم جاهزية المؤسسة للسحابة وتحسينها. يوصي الإطار بمراحل رحلة متكررة للتحول الرقمي تشمل التصور، المواءمة، الإطلاق، وتوسيع النطاق. يساعد AWS CAF المؤسسات على تحديد فرص التحول الرقمي، وتقييم استعدادها السحابي، وتطوير خارطة طريق للتحول بشكل تكراري، مما يؤدي إلى تسريع نتائج الأعمال مثل خفض التكاليف وزيادة المرونة والابتكار وتحسين تجربة العملاء والموظفين.
</summary>


In [24]:
import bs4


def extract_tag(xml_text, tag='summary'):
    soup = bs4.BeautifulSoup(xml_text, 'html.parser')
    summary_tag = soup.find(tag)
    if summary_tag:
        return summary_tag.text.strip()
    else:
        return None

In [26]:
print(extract_tag(response_body["content"][0]["text"], tag='summary'))

يقدم إطار عمل اعتماد السحابة من AWS (AWS CAF) إرشادات حول أفضل الممارسات لمساعدة المؤسسات على التحول الرقمي باستخدام خدمات AWS. يحدد الإطار 6 مناظير رئيسية (الأعمال، الأشخاص، الحوكمة، المنصات، الأمان، العمليات) و إمكانيات تأسيسية تحت كل منظور لتقييم جاهزية المؤسسة للسحابة وتحسينها. يوصي الإطار بمراحل رحلة متكررة للتحول الرقمي تشمل التصور، المواءمة، الإطلاق، وتوسيع النطاق. يساعد AWS CAF المؤسسات على تحديد فرص التحول الرقمي، وتقييم استعدادها السحابي، وتطوير خارطة طريق للتحول بشكل تكراري، مما يؤدي إلى تسريع نتائج الأعمال مثل خفض التكاليف وزيادة المرونة والابتكار وتحسين تجربة العملاء والموظفين.
